## Performance comparison

In this notebook, we measure the performance gain between pure Python methods and compiled Python methods.

To compile the desired module, the [Pythran](https://github.com/serge-sans-paille/pythran) module was used. It converts Python methods to C++11 templated methods and compiles them. It is advised to optimise the compilation for the local architecture, even if the default compilation parameters already yield significant improvements in execution time. For example, we can easily get multi-threaded methods using OpenMP instructions. For more details, see [pythran documentation](http://pythran.readthedocs.io).

In [1]:
import numpy as np
import cv2
from skimage.measure import compare_ssim, compare_mse, compare_psnr

from utils.super_resolution_utils import *
from utils import cpp_grad_utils
from utils import cpp_grad_utils_compiled

import time

In [2]:
# Open original image and extract region of interest
im_bgr = cv2.imread('hedgehog.jpg')
im = im_bgr.astype(np.float)

roi = im[500:900, 800:1300]
#roi = im

# Set variables used to generate low resolution images
q = 2
nb_lr_im = 8
noise = 0.1*(np.max(roi) - np.min(roi))

# Set variables for super resolution
iterations = 200
l = 6
beta = 0.8
dt = 1/(np.abs(l)*max(4/beta, 2))
version_tau = 1
C = 1/6
exp_file = 'experiences.txt'

In [3]:
# Create small images and store them somewhere on disk. Should work for gray and color images
lr_images = createLRSamples(roi, nb_lr_images=nb_lr_im, q=q, noise_variance=noise)

for i in range(lr_images.shape[0]):
    cv2.imwrite('lr_images/lr_'+str(i)+'.png', lr_images[i])
cv2.imwrite('roi.png', roi)

# Create new empty HR image. Should work for color and gray images
hr_dims = get_hr_dims(lr_images.shape, q)
hr_image = np.zeros(hr_dims)

comparison_gradient = 0
denoising_gradient = 0
smoothing_gradient = 0
tau = 0

In [4]:
# Begins the experiment report
with open(exp_file, 'a') as f:
    f.write("#Conditions de l'expérience:\n")
    f.write("#q={}, nb_lr_images={}, dt = {}, lambda={}, beta={}, nb_iterations={}, version_tau={}, C={}\n".format(q, nb_lr_im, dt, l, beta, iterations, version_tau, C))

In [5]:
t1 = time.time()

for iteration in range(iterations+1):

    # Save result
    if iteration % 5 == 0:
        mse = compare_mse(roi, hr_image)
        ssim = compare_ssim(roi, hr_image, multichannel=True)
        psnr = compare_psnr(roi, hr_image, data_range=255.0)
        name = 'hr_images/hr_'+str(iteration)+'.png'
        cv2.imwrite(name, hr_image)
        with open(exp_file, 'a') as f:
            f.write("{}\t{}\t{}\t{}\t{}\t{}\n".format(time.time() - t1, iteration, mse, ssim, psnr, name))
            if(mse > 1e8):
                f.write("Experience diverging, stopping now\n")
                raise ValueError("The algorithm is diverging")
            if(np.isnan(mse) or np.isnan(ssim) or np.isnan(psnr)):
                f.write("Computation error\n")
                raise ValueError("Computation error")


    # Compute directional order 1 and 2 gradients
    ### In order to use the uncompiled methods, replace cpp_grad_utils_compiled by cpp_grad_utils
    Ix = cpp_grad_utils_compiled.grad_x(hr_image)
    Iy = cpp_grad_utils_compiled.grad_y(hr_image)
    Ixy = cpp_grad_utils_compiled.grad_y(Ix)
    Ixx = cpp_grad_utils_compiled.grad_xx(hr_image)
    Iyy = cpp_grad_utils_compiled.grad_yy(hr_image)

    #Compute data fidelity energy gradient, denoising energy gradient, smoothing energy gradient
    comparison_gradient = data_fidelity_gradient_2(hr_image, lr_images, q)
    denoising_gradient = cpp_grad_utils_compiled.TV_regularization(Ix, Iy, Ixy, Ixx, Iyy, beta)
    smoothing_gradient = cpp_grad_utils_compiled.heat_gradient(Ix, Iy, Ixy, Ixx, Iyy)

    #Compute weighting factor
    if version_tau == 1:
        tau = cpp_grad_utils_compiled.compute_tau_1(Ix, Iy, C)
    elif version_tau == 2:
        tau = cpp_grad_utils_compiled.compute_tau_2(Ix, Iy)
    elif version_tau == 3:
        tau = cpp_grad_utils_compiled.compute_tau_3(Ix, Iy, C)

    #Update high resolution image
    hr_image -= dt*(comparison_gradient) + l*(tau*denoising_gradient + (1-tau)*smoothing_gradient)

ValueError: The algorithm is diverging

In [6]:
63/4.0


15.75